In [1]:
import numpy as np
import json
import smplx
import torch

In [2]:
smplxs_data = np.load('./smplxs.npy', allow_pickle=True)
smplxs_data

array([{'betas': array([-0.72901833, -0.02397307,  0.36086836,  0.09754749, -0.01564887,
               0.1988453 , -0.2623446 , -0.02810438,  0.0090983 , -0.01106459],
             dtype=float32), 'expression': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), 'fullpose': array([[-3.04492068e+00,  2.83338800e-02,  3.65486562e-01],
              [-1.75652146e-01,  2.87480533e-01, -2.83291727e-01],
              [ 3.53024781e-01, -1.10754657e+00,  1.73701197e-02],
              [ 5.65524995e-02, -1.73291564e-01, -3.17672938e-01],
              [-7.07815634e-03,  8.51745307e-02,  2.33980224e-01],
              [-1.54024944e-01, -2.43327945e-01, -7.95316994e-02],
              [ 1.33439079e-01, -7.93264881e-02,  3.55536938e-01],
              [-8.55945349e-02,  2.40227237e-01, -2.18611062e-01],
              [ 1.16125286e+00, -3.82055372e-01,  1.75869480e-01],
              [-5.14477968e-01,  3.04610878e-01,  1.29065141e-01],
              [-2.47168332e-01, -3.28460112e-02, 

In [3]:
smplx_data = smplxs_data[77]
smplx_data

{'betas': array([-0.72901833, -0.02397307,  0.36086836,  0.09754749, -0.01564887,
         0.1988453 , -0.2623446 , -0.02810438,  0.0090983 , -0.01106459],
       dtype=float32),
 'expression': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'fullpose': array([[-3.20897079e+00, -1.20439276e-01,  7.25955486e-01],
        [ 2.71920376e-02,  4.41980034e-01, -2.40042552e-01],
        [ 5.93240738e-01, -7.50848174e-01, -7.10298195e-02],
        [ 2.85282820e-01, -7.00715035e-02,  9.87915620e-02],
        [ 2.87207775e-02,  3.13295990e-01,  3.41282994e-01],
        [-2.20449743e-04, -4.56348538e-01, -6.85096383e-02],
        [ 5.88779807e-01, -2.06810296e-01, -1.00218788e-01],
        [-1.53893754e-01,  6.74618594e-03, -3.76236528e-01],
        [ 9.27366555e-01,  6.71612263e-01, -2.29039490e-02],
        [-7.36776412e-01, -8.27886537e-02,  2.04084560e-01],
        [-5.79222679e-01, -4.88087311e-02,  5.00915170e-01],
        [ 3.21417749e-01,  1.49656340e-01, -3.06734383e-01]

In [4]:
model_folder = '/home/yeyiqi/Documents/repos/GaussianAvatar/assets/smpl_files'
model_type = 'smplx'
kwargs = dict(gender='neutral',
        num_betas=10,
        use_face_contour=True,
        flat_hand_mean=False,
        use_pca=False,
        batch_size=1,
        ext = 'pkl'
)

model = smplx.create(
    model_path = model_folder,
    model_type = model_type,
    **kwargs
)

/home/yeyiqi/.conda/envs/MyFirst/lib/python3.10/site-packages/smplx/body_models.py:1009: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  dynamic_lmk_bary_coords = torch.tensor(


In [5]:
smplx_shape = {'betas': (-1, 10), 'transl': (-1, 3), 'global_orient': (-1, 3), 
        'body_pose': (-1, 21, 3), 'left_hand_pose': (-1, 15, 3), 'right_hand_pose': (-1, 15, 3), 
        'leye_pose': (-1, 3), 'reye_pose': (-1, 3), 'jaw_pose': (-1, 3), 'expression': (-1, 10)}

In [6]:
body_model_param = {
    'betas': np.expand_dims(smplx_data['betas'],0),
    'expression': np.expand_dims(smplx_data['expression'],0),
    'transl': np.expand_dims(smplx_data['transl'],0),
    'global_orient': np.expand_dims(smplx_data['fullpose'][0], 0),
    'body_pose': np.expand_dims(smplx_data['fullpose'][1:22], 0),
    'jaw_pose': np.expand_dims(smplx_data['fullpose'][22], 0),
    'leye_pose': np.expand_dims(smplx_data['fullpose'][23], 0),
    'reye_pose': np.expand_dims(smplx_data['fullpose'][24], 0),
    'left_hand_pose': np.expand_dims(smplx_data['fullpose'][25:40], 0),
    'right_hand_pose': np.expand_dims(smplx_data['fullpose'][40:], 0)
}

In [7]:
body_model_param_tensor = {
    key: torch.tensor(body_model_param[key]) for key in body_model_param.keys()
}

In [8]:
output = model(**body_model_param_tensor)
vertices = output.vertices.detach().cpu().numpy().squeeze()
joints = output.joints.detach().cpu().numpy().squeeze()
faces = model.faces

torch.Size([1, 55, 3, 3])


In [9]:
with open('smplx_77.ply', 'w') as file:
    file.write(f'ply\n')
    file.write(f"format ascii 1.0\n")
    file.write(f"element vertex {len(vertices)}\n")
    file.write(f"property float x\n")
    file.write(f"property float y\n")
    file.write(f"property float z\n")
    file.write(f"element face {len(faces)}\n")
    file.write(f"property list uchar int vertex_indices\n")
    file.write(f"end_header\n")
    for i in range(len(vertices)):
        vertex = vertices[i]
        file.write(f"{vertex[0]} {vertex[1]} {vertex[2]}\n")
    for face in faces:
        file.write(f"3 {face[0]} {face[1]} {face[2]}\n")

In [10]:
smplx_neutral_path = '/home/yeyiqi/Documents/models/SMPLX/models_smplx_v1_1/models/smplx/SMPLX_NEUTRAL.npz'
smplx_neutral = np.load(smplx_neutral_path, allow_pickle=True)
weights = smplx_neutral["weights"]

In [16]:
smplx_neutral.files

['hands_meanr',
 'hands_meanl',
 'lmk_bary_coords',
 'vt',
 'posedirs',
 'part2num',
 'hands_coeffsr',
 'lmk_faces_idx',
 'J_regressor',
 'dynamic_lmk_faces_idx',
 'hands_componentsr',
 'shapedirs',
 'dynamic_lmk_bary_coords',
 'ft',
 'hands_componentsl',
 'joint2num',
 'v_template',
 'allow_pickle',
 'f',
 'hands_coeffsl',
 'kintree_table',
 'weights']

In [21]:
smplx_neutral['posedirs'].shape

(10475, 3, 486)

In [20]:
weights.shape

(10475, 55)

In [12]:
np.save("./weights.npy", weights)

In [13]:
weights = np.load("weights.npy")

In [14]:
weights

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00181066, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)